## High-Performance Pandas: eval() and query()

### Motivating query() and eval(): Compound Expressions

In [5]:
import numpy as np
rng = np.random.RandomState(42)
# Need to typecast the range into an int value else TypeError
x = np.random.rand(int(1E6))
y = np.random.rand(int(1E6))

%timeit x + y

4.29 ms ± 924 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

396 ms ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# The expression below
mask = (x > 0.5) & (y < 0.5)

# is equivalent to
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

# ie. every intermediate step is explicitly allocated in memory

In [13]:
!pip install numexpr

In [14]:
# Further reference for np.allclose function: https://numpy.org/doc/stable/reference/generated/numpy.allclose.html
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

### pandas.eval() for Efficient Operations

In [15]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                     for i in range(4))

In [16]:
%timeit df1 + df1 + df3 + df4

98.5 ms ± 7.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
# eval is about 50% faster
%timeit pd.eval('df1 + df2 + df3 + df4')

50.7 ms ± 994 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
np.allclose(df1 + df2 + df3 + df4, pd.eval('df1 + df2 + df3 + df4'))

True

#### Operations supported by pd.eval()

In [19]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                                       for i in range(5))

#### Arithmetic operators

In [20]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

#### Comparison operators

In [22]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### Bitwise operators

In [23]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [24]:
# Supports also literal expressions
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### Objects attributes and indices

In [25]:
# Also supports indexing
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

In [26]:
# loops and other constructs are still not supported!

### DataFrame.eval() for Column-Wise Operations

In [27]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [28]:
result1 = (df['A'] + df['B']) / (df['C'] -1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

In [29]:
# column names are treated as variables within the eval expression
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

#### Assignment in DataFrame.eval()

In [30]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [31]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


In [32]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


#### Local variables in DataFrame.eval()

In [34]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

### DataFrame.query() Method

In [35]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

In [36]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In [37]:
# Also accepts local variables
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

### Performance: When to Use These Functions

In [38]:
x = df[(df.A < 0.5) & (df.B < 0.5)]

In [39]:
df.values.nbytes

32000